In [ ]:
cd ../

In [ ]:
from Lib.WebScrapping.CorpusScraper import *
from Lib.TextAnalytics.Methods import *
import sys
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
webpages = get_tag_corpus(["hazard", "battery", "marine"], pages=1, methods=['Scholar'], from_year=2015)

In [ ]:
from Lib.PreProcessing.General import *
corpus = [remove_stop_words(webpage.textual_content, 'Resources/stop-words.txt') for webpage in webpages]

In [ ]:
for document in corpus:
    print(len(document))

In [ ]:
result = get_tfidf(corpus)

In [ ]:
features = result[1].get_feature_names_out()

In [ ]:
non_zero_results = [csr_matrix.nonzero() for csr_matrix in result[0]]

In [ ]:
non_zero_result_importance = list()
for i, non_zero_result in enumerate(non_zero_results):
    temp = list()
    for (row, col) in zip(*non_zero_result):
        temp.append(((row, col), result[0][i][row, col]))
    temp.sort(key=lambda a: a[1], reverse=True)
    non_zero_result_importance.append(temp)

In [ ]:
non_zero = np.empty((len(features), len(webpages)), dtype=float)
for idx, non_zero_result in enumerate(non_zero_result_importance):
    for ((row, col), importance) in non_zero_result:
        non_zero[col, idx] = importance
non_zero = np.nan_to_num(non_zero, posinf=1, neginf=0)

In [ ]:
df = pd.DataFrame(non_zero, index=features, columns=[webpage.title for webpage in webpages])

In [ ]:
for webpage in webpages:
    print(webpage.title, webpage.url)

In [ ]:
threshold = 0.2
top_keywords = df.loc[(df > threshold).any(axis=1), (df > threshold).any()]
top_keywords
#top_keywords.style.background_gradient()